# 🌍 Mwalimu-LangLearn: Interactive Language Learning Tool with Gemma 3n

**An Educational Tool for Multilingual Language Learning using Google's Gemma 3n**

---

## Overview

This notebook builds an interactive language learning application powered by **Google's Gemma 3n** model. The app leverages Gemma 3n's multimodal capabilities (text + image) and multilingual support (140+ languages) to create a comprehensive language learning experience.

### Features:
1. **📝 Writing Exercise Feedback** — Real-time grammar, vocabulary, and style correction
2. **🖼️ Visual Object Recognition** — Upload images to identify objects and learn translations
3. **🔄 Translation Practice** — Translate sentences and get detailed feedback
4. **📚 Vocabulary Builder** — Generate contextual vocabulary lessons
5. **💬 Conversation Practice** — Simulate dialogues in the target language

### Why Gemma 3n?
- **Compact & Efficient**: E2B (effective 2B params) or E4B (effective 4B params) — runs on Kaggle free GPUs
- **Multimodal**: Handles text + images natively (MobileNet-V5 vision encoder)
- **Multilingual**: Trained on 140+ languages — perfect for African language contexts
- **On-device ready**: Designed for edge deployment (phones, tablets)

### Architecture
```
┌─────────────────────────────────────────────────┐
│                 Gradio Interface                  │
│  ┌──────────┬──────────┬──────────┬───────────┐  │
│  │ Writing  │  Visual  │ Translate│  Vocab    │  │
│  │ Feedback │  Learn   │ Practice │  Builder  │  │
│  └────┬─────┴────┬─────┴────┬─────┴─────┬────┘  │
│       │          │          │           │        │
│  ┌────▼──────────▼──────────▼───────────▼────┐   │
│  │        Gemma 3n E2B/E4B (Instruct)        │   │
│  │   AutoModelForImageTextToText + Processor  │   │
│  │   ┌─────────┐  ┌──────────┐  ┌─────────┐ │   │
│  │   │  Text   │  │  Vision  │  │  Audio  │ │   │
│  │   │ Decoder │  │ Encoder  │  │ Encoder │ │   │
│  │   └─────────┘  └──────────┘  └─────────┘ │   │
│  └───────────────────────────────────────────┘   │
└─────────────────────────────────────────────────┘
```

---

**Author**: Pascal Burume Buhendwa  
**Environment**: Kaggle Notebook (GPU T4 x2 or P100)  
**Model**: Google Gemma 3n E2B-IT (Instruction Tuned)  
**Framework**: HuggingFace Transformers + Gradio

---
## Step 1: Environment Setup & Dependencies

We need to install the latest versions of key libraries:
- **transformers** (≥4.53.0) — Required for Gemma 3n support
- **timm** — For the MobileNet-V5 vision encoder
- **accelerate** — For efficient model loading with `device_map="auto"`
- **gradio** — For the interactive web interface
- **kagglehub** — To download models from Kaggle

In [1]:
# ============================================================
# STEP 1: Install Dependencies
# ============================================================
# Install the latest transformers (Gemma 3n requires >=4.53.0)
!pip install -q git+https://github.com/huggingface/transformers.git

# Install timm for vision encoder (MobileNet-V5)
!pip install -q timm --upgrade

# Install accelerate for efficient model distribution
!pip install -q accelerate

# Install Gradio for the interactive UI
!pip install -q gradio

# Install kagglehub for model download
!pip install -q kagglehub

# Install Pillow for image handling
!pip install -q Pillow

# Install Update
!pip install --upgrade pip

print("✅ All dependencies installed successfully!")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
✅ All dependencies installed successfully!


---
## Step 2: Import Libraries

Import all required Python packages. Key classes:
- `AutoProcessor` — Handles tokenization for both text and image inputs
- `AutoModelForImageTextToText` — The multimodal Gemma 3n model class
- `kagglehub.model_download` — Downloads model weights from Kaggle

In [2]:
# ============================================================
# STEP 2: Import Libraries
# ============================================================
import os
import torch
import kagglehub
import gradio as gr
from time import time
from PIL import Image
from transformers import (
    AutoProcessor,
    AutoModelForImageTextToText,
)

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected. Model will run on CPU (much slower).")

PyTorch version: 2.9.0+cu128
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.6 GB


---
## Step 3: Download and Load the Gemma 3n Model

### How the model loading works:

1. **`kagglehub.model_download()`** downloads the model weights from Kaggle's model hub to a local cache
2. **`AutoProcessor.from_pretrained()`** loads the tokenizer + image processor that converts raw text/images into tensors
3. **`AutoModelForImageTextToText.from_pretrained()`** loads the actual model with:
   - `torch_dtype="auto"` — Automatically selects the best precision (bfloat16 on GPU)
   - `device_map="auto"` — Distributes model layers across available GPUs/CPU automatically

### Model Variants:
| Variant | Effective Params | Total Params | Best For |
|---------|-----------------|-------------|----------|
| `gemma-3n-e2b-it` | 2B | ~6B | Kaggle free GPU, fast inference |
| `gemma-3n-e4b-it` | 4B | ~8B | Better quality, needs more VRAM |

We use **E2B-IT** (instruction-tuned) for efficiency on Kaggle's free T4 GPU.

In [3]:
!pip install kagglehub transformers torch accelerate

In [ ]:
import os
import kagglehub
from transformers import AutoProcessor, AutoModelForImageTextToText

# Set your Kaggle credentials (these are the correct ones!)
os.environ['KAGGLE_USERNAME'] = 'Your Kaggle Username'
os.environ['KAGGLE_KEY'] = 'Your Kaggle API Key'

print("📥 Downloading Gemma 3n E2B-IT from Kaggle...")
GEMMA_PATH = kagglehub.model_download(
    "google/gemma-3n/transformers/gemma-3n-e2b-it"
)
print(f"✅ Model downloaded to: {GEMMA_PATH}")

# Load the processor
print("🔧 Loading processor...")
processor = AutoProcessor.from_pretrained(GEMMA_PATH)

# Load the model
print("🧠 Loading model (this may take 2-5 minutes)...")
model = AutoModelForImageTextToText.from_pretrained(
    GEMMA_PATH,
    torch_dtype="auto",
    device_map="auto",
)

print(f"✅ Model loaded successfully!")
print(f"   Device: {model.device}")
print(f"   Dtype: {model.dtype}")

📥 Downloading Gemma 3n E2B-IT from Kaggle...


100%|██████████| 16.0G/16.0G [03:42<00:00, 76.9MB/s]

Extracting files...


✅ Model downloaded to: /root/.cache/kagglehub/models/google/gemma-3n/transformers/gemma-3n-e2b-it/2
🔧 Loading processor...
🧠 Loading model (this may take 2-5 minutes)...


Loading weights:   0%|          | 0/1556 [00:00<?, ?it/s]

✅ Model loaded successfully!
   Device: cuda:0
   Dtype: torch.bfloat16


---
## Step 4: Core Inference Functions

We build two core functions:

### 4a. `query_model()` — Text-only inference
This function:
1. Constructs a **chat-template** message with system + user roles
2. Applies the chat template via `processor.apply_chat_template()`
3. Tokenizes and sends to model device
4. Generates response with `model.generate()`
5. Decodes and extracts only the assistant's response

### 4b. `query_model_with_image()` — Multimodal inference (text + image)
This function:
1. Opens and preprocesses the image via PIL
2. Constructs a message with an `{"type": "image"}` content block
3. The processor automatically resizes the image to 256/512/768px and encodes it to 256 tokens
4. Both image and text tokens are sent to the model together

In [5]:
# ============================================================
# STEP 4: Core Inference Functions
# ============================================================

def query_model(prompt, system_prompt=None, max_new_tokens=512):
    """
    Send a text-only query to Gemma 3n using chat template format.

    Args:
        prompt (str): The user's question or instruction
        system_prompt (str): Optional system instruction for the model
        max_new_tokens (int): Maximum length of generated response

    Returns:
        tuple: (response_text, execution_time_seconds)
    """
    start_time = time()

    # Build the conversation messages
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    # Apply chat template — this adds special tokens like <start_of_turn>
    input_text = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize and move to model device
    inputs = processor(
        text=input_text,
        return_tensors="pt"
    ).to(model.device, dtype=model.dtype)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            disable_compile=True,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    # Decode the full output
    full_text = processor.batch_decode(
        outputs,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )[0]

    # Extract only the assistant's response (after the prompt)
    # The model echoes the input, so we split on the last user message
    response = full_text.split(prompt)[-1].strip()

    total_time = round(time() - start_time, 2)
    return response, total_time


def query_model_with_image(image, prompt, system_prompt=None, max_new_tokens=512):
    """
    Send a multimodal query (image + text) to Gemma 3n.

    The vision encoder (MobileNet-V5) processes the image at
    256x256, 512x512, or 768x768 resolution, encoding it to 256 tokens.

    Args:
        image: PIL Image or file path
        prompt (str): Text instruction about the image
        system_prompt (str): Optional system instruction
        max_new_tokens (int): Maximum response length

    Returns:
        tuple: (response_text, execution_time_seconds)
    """
    start_time = time()

    # Ensure we have a PIL Image
    if isinstance(image, str):
        image = Image.open(image)
    if image.mode != "RGB":
        image = image.convert("RGB")

    # Build multimodal messages with image content block
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({
        "role": "user",
        "content": [
            {"type": "image", "image": image},  # Image content block
            {"type": "text", "text": prompt},    # Text instruction
        ]
    })

    # Apply chat template
    input_text = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Process both text and image together
    inputs = processor(
        text=input_text,
        images=image,
        return_tensors="pt"
    ).to(model.device, dtype=model.dtype)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            disable_compile=True,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    # Decode response
    full_text = processor.batch_decode(
        outputs,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )[0]

    # Extract assistant response
    response = full_text.split(prompt)[-1].strip()

    total_time = round(time() - start_time, 2)
    return response, total_time


# Quick test
print("🧪 Testing text-only inference...")
test_response, test_time = query_model(
    "Translate 'Hello, how are you?' to French.",
    system_prompt="You are a helpful language tutor. Be concise."
)
print(f"   Response: {test_response}")
print(f"   Time: {test_time}s")

🧪 Testing text-only inference...
   Response: model
Bonjour, comment allez-vous? 

(Formal)

or

Salut, comment vas-tu? 

(Informal)
   Time: 10.6s


---
## Step 5: Language Learning Feature Functions

Each function maps to a specific learning mode in the app. They all use the core `query_model()` or `query_model_with_image()` functions but with specialized system prompts that instruct Gemma 3n to act as a language tutor.

### Supported Language Pairs:
Gemma 3n supports 140+ languages. Key languages for our context:
- **English** ↔ **French** (DRC official language)
- **English** ↔ **Swahili** (Eastern DRC lingua franca)
- **French** ↔ **Japanese** (for ABE Initiative students)
- **English** ↔ **Lingala** (Western DRC lingua franca)
- And many more...

In [6]:
# ============================================================
# STEP 5: Language Learning Feature Functions
# ============================================================

# Available target languages
LANGUAGES = [
    "French", "Swahili", "Japanese", "English", "Lingala",
    "Spanish", "Portuguese", "Arabic", "Chinese (Mandarin)",
    "German", "Korean", "Hindi", "Italian", "Russian"
]

PROFICIENCY_LEVELS = ["Beginner (A1-A2)", "Intermediate (B1-B2)", "Advanced (C1-C2)"]


# ──────────────────────────────────────────────────────────
# FEATURE 1: Writing Exercise Feedback
# ──────────────────────────────────────────────────────────
def writing_feedback(text, target_language, proficiency_level):
    """
    Analyze written text in the target language and provide
    grammar corrections, vocabulary suggestions, and style tips.
    """
    if not text.strip():
        return "⚠️ Please enter some text to analyze.", ""

    system_prompt = f"""You are an expert {target_language} language tutor.
The student's proficiency level is {proficiency_level}.
Analyze their writing and provide:
1. **Corrections**: Fix grammar, spelling, and syntax errors. Show the corrected version.
2. **Explanations**: Explain each error briefly so the student learns.
3. **Vocabulary**: Suggest better word choices or useful alternatives.
4. **Style Tips**: Provide natural phrasing suggestions.
5. **Score**: Give a score out of 10 with encouragement.
Be encouraging and supportive. Use examples when helpful."""

    prompt = f"""Please review and correct this {target_language} text written by a {proficiency_level} student:

\"\"\"
{text}
\"\"\"

Provide detailed feedback with corrections and explanations."""

    response, exec_time = query_model(prompt, system_prompt, max_new_tokens=1024)
    return response, f"⏱️ Response time: {exec_time}s"


# ──────────────────────────────────────────────────────────
# FEATURE 2: Visual Object Recognition & Translation
# ──────────────────────────────────────────────────────────
def visual_learning(image, target_language, source_language="English"):
    """
    Identify objects in an uploaded image and provide translations
    with pronunciation guides and example sentences.
    """
    if image is None:
        return "⚠️ Please upload an image.", ""

    system_prompt = f"""You are a visual language learning assistant.
Help students learn {target_language} vocabulary through images.
Be thorough but organized in your responses."""

    prompt = f"""Look at this image carefully. Please:

1. **Identify** all major objects, people, and elements visible in the image.
2. **Translate** each identified item from {source_language} to {target_language}.
3. **Pronunciation**: Include pronunciation guide for each {target_language} word.
4. **Example Sentences**: Write 2-3 simple sentences in {target_language} using these vocabulary words.
5. **Cultural Note**: If relevant, share any cultural context about these items in {target_language}-speaking regions.

Format the response clearly with sections."""

    response, exec_time = query_model_with_image(
        image, prompt, system_prompt, max_new_tokens=1024
    )
    return response, f"⏱️ Response time: {exec_time}s"


# ──────────────────────────────────────────────────────────
# FEATURE 3: Translation Practice with Feedback
# ──────────────────────────────────────────────────────────
def translation_practice(original_text, student_translation, source_lang, target_lang, proficiency):
    """
    The student translates a sentence, and the model provides
    detailed feedback comparing their translation to an ideal one.
    """
    if not original_text.strip() or not student_translation.strip():
        return "⚠️ Please provide both the original text and your translation.", ""

    system_prompt = f"""You are a professional {source_lang}-to-{target_lang} translation tutor.
The student's level is {proficiency}. Be encouraging but thorough."""

    prompt = f"""A {proficiency} student was asked to translate this from {source_lang} to {target_lang}:

**Original ({source_lang}):** \"{original_text}\"
**Student's Translation ({target_lang}):** \"{student_translation}\"

Please provide:
1. **Your ideal translation** of the original text
2. **Accuracy score** (1-10)
3. **Specific corrections** — What did the student get wrong?
4. **What they got right** — Praise good parts
5. **Grammar notes** — Key grammar rules they should review
6. **Alternative translations** — Other valid ways to translate this"""

    response, exec_time = query_model(prompt, system_prompt, max_new_tokens=1024)
    return response, f"⏱️ Response time: {exec_time}s"


# ──────────────────────────────────────────────────────────
# FEATURE 4: Vocabulary Builder
# ──────────────────────────────────────────────────────────
def vocabulary_builder(topic, target_language, proficiency, num_words=10):
    """
    Generate a themed vocabulary lesson with words, translations,
    example sentences, and a mini-quiz.
    """
    if not topic.strip():
        return "⚠️ Please enter a topic.", ""

    system_prompt = f"""You are a {target_language} vocabulary tutor.
Create engaging vocabulary lessons appropriate for {proficiency} learners.
Make learning fun and memorable."""

    prompt = f"""Create a vocabulary lesson about \"{topic}\" in {target_language} for a {proficiency} student.

Include:
1. **{num_words} Key Words**: Each with translation, pronunciation, and part of speech
2. **Example Sentences**: One sentence per word showing it in context
3. **Common Phrases**: 3-5 useful phrases related to the topic
4. **Mini Quiz**: 3 fill-in-the-blank questions to test the vocabulary
5. **Memory Tips**: Mnemonics or tricks to remember difficult words"""

    response, exec_time = query_model(prompt, system_prompt, max_new_tokens=1500)
    return response, f"⏱️ Response time: {exec_time}s"


# ──────────────────────────────────────────────────────────
# FEATURE 5: Conversation Practice
# ──────────────────────────────────────────────────────────
def conversation_practice(user_message, scenario, target_language, proficiency, chat_history=""):
    """
    Simulate a conversation in the target language.
    The model plays a character and provides corrections inline.
    """
    if not user_message.strip():
        return "⚠️ Please type a message to continue the conversation.", "", ""

    system_prompt = f"""You are a friendly conversation partner helping a {proficiency} student
practice {target_language}. The scenario is: {scenario}.

Rules:
- Respond naturally IN {target_language}
- After your response, add a section called "📝 Language Notes:" where you:
  * Correct any errors in the student's message
  * Explain grammar points
  * Suggest more natural ways to say things
- Keep the conversation going by asking a follow-up question
- Adjust complexity to {proficiency} level"""

    # Include chat history for context
    prompt = f"""{f'Previous conversation:{chr(10)}{chat_history}{chr(10)}{chr(10)}' if chat_history else ''}Student says: \"{user_message}\"

Respond in character (in {target_language}), then provide language notes."""

    response, exec_time = query_model(prompt, system_prompt, max_new_tokens=768)

    # Update chat history
    updated_history = f"{chat_history}\nStudent: {user_message}\nTutor: {response}"

    return response, updated_history, f"⏱️ Response time: {exec_time}s"


print("✅ All feature functions defined!")

✅ All feature functions defined!


---
## Step 6: Quick Feature Tests (Before Building UI)

Let's verify each feature works before building the Gradio interface.

In [7]:
# ============================================================
# STEP 6: Test Each Feature
# ============================================================

# Test 1: Writing Feedback (French)
print("="*60)
print("TEST 1: Writing Feedback (French)")
print("="*60)
result, timing = writing_feedback(
    "Je suis allé au magazin pour acheter du pain. Le pain était très bon et je mange beaucoup.",
    "French",
    "Intermediate (B1-B2)"
)
print(result)
print(timing)
print()

TEST 1: Writing Feedback (French)
model
Bonjour! C'est très bien que tu aies écrit ce court texte en français, c'est un excellent début!  It's great that you're practicing!  Let's take a look at it and see how we can make it even better.

Here's the analysis and corrections:

**Original Text:**

"""
Je suis allé au magazin pour acheter du pain. Le pain était très bon et je mange beaucoup.
"""

**Corrected Text:**

"""
Je suis allé au magasin pour acheter du pain. Le pain était très bon, et je mange beaucoup.
"""

**1. Corrections:**

*   **"magazin" changed to "magasin"**:  "Magasin" is the correct form of the word for "shop" or "store" in French.  "Magazin" is a misspelling.
*   **Added a comma after "bon"**:  The sentence is a simple statement, and a comma is needed to separate the two independent clauses ("Le pain était très bon" and "je mange beaucoup").

**2. Explanations:**

*   **Spelling:**  The most obvious error was the misspelling of "magasin."  Pay attention to spelling!  I

In [8]:
# Test 2: Vocabulary Builder (Swahili)
print("="*60)
print("TEST 2: Vocabulary Builder (Swahili)")
print("="*60)
result, timing = vocabulary_builder(
    "Food and Cooking",
    "Swahili",
    "Beginner (A1-A2)",
    num_words=5
)
print(result)
print(timing)

TEST 2: Vocabulary Builder (Swahili)
model
## Swahili Vocabulary Lesson: Food and Cooking (A1-A2)

**Welcome, mwanafunzi! (Student!)**  Let's learn some Swahili words about food and cooking!  This lesson will help you understand what people say when talking about what they eat and how they prepare it.  Let's have fun!

**1. Key Words:**

| **Swahili Word** | **Translation** | **Pronunciation** | **Part of Speech** |
|---|---|---|---|
| **chakula** | food | cha-ku-la | Noun (general term for food) |
| **chakula chakupikia** | cooking food | cha-ku-la cha-ku-pi-kia | Noun (food used for cooking) |
| **nyama** | meat | na-ma | Noun (meat) |
| **lazima** | must/have to | la-zi-ma | Adverb (must/have to) |
| **kupika** | to cook | ku-pi-ka | Verb (to cook) |

**2. Example Sentences:**

*   **chakula:**  "Ninapenda chakula cha safi." (I like clean food.)
*   **chakula chakupikia:** "Tunapika chakula cha kupikia kwa nyama." (We cook food for cooking with meat.)
*   **nyama:** "Mimi ninapenda 

In [9]:
# Test 3: Translation Practice (English → Japanese)
print("="*60)
print("TEST 3: Translation Practice (English → Japanese)")
print("="*60)
result, timing = translation_practice(
    "The weather is very nice today, I want to go for a walk.",
    "今日の天気はとてもいいです。散歩に行きたいです。",
    "English",
    "Japanese",
    "Beginner (A1-A2)"
)
print(result)
print(timing)

TEST 3: Translation Practice (English → Japanese)
model
Okay, let's take a look at your translation! You've done a great job! Here's a breakdown to help you understand and improve.

**1. Ideal Translation:**

The most natural and common way to say this in Japanese would be:

**今日はいい天気だから、散歩に行きたいな。**
(Kyou wa ii tenki dakara, sanpo ni ikitai na.)

*   `今日は` (Kyou wa) - Today
*   `いい天気` (ii tenki) - Good weather
*   `だから` (dakara) - Because / Therefore
*   `散歩に` (sanpo ni) - For a walk (dative case)
*   `行きたいな` (ikitai na) - I want to go (a softer, more casual expression)

**2. Accuracy Score:**

I'd give your translation a **9/10**. It's very close and perfectly understandable!

**3. Specific Corrections:**

Your translation, "今日の天気はとてもいいです。散歩に行きたいです。" is grammatically correct and conveys the meaning. However, there's a slight difference in nuance.  The difference is in the *reason* for wanting to go for a walk.

*   **"今日の天気はとてもいいです。" (Kyou wa totemo ii desu.)** - "Today the weather is

---
## Step 7: Build the Gradio Interface

Now we build the full interactive UI using **Gradio Blocks**. The interface has 5 tabs, one for each learning mode.

### Gradio Architecture:
- **`gr.Blocks`** — The main container for custom layouts
- **`gr.Tabs`** — Creates a tabbed interface
- **`gr.Row/Column`** — Layout controls
- Each tab connects its button click to the corresponding feature function

In [10]:
# ============================================================
# STEP 7: Build the Gradio Interface
# ============================================================

# Custom CSS for better styling
custom_css = """
.gradio-container {
    max-width: 1200px !important;
}
.feature-description {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 15px;
    border-radius: 10px;
    margin-bottom: 15px;
}
"""

# Build the interface
with gr.Blocks(
    title="🌍 Mwalimu-LangLearn: AI Language Tutor",
    css=custom_css,
    theme=gr.themes.Soft()
) as demo:

    # ── Header ────────────────────────────────────────────
    gr.Markdown("""
    # 🌍 Mwalimu-LangLearn: Interactive Language Learning Tool
    ### Powered by Google Gemma 3n — Multilingual AI Tutor

    Learn any of **140+ languages** with AI-powered feedback on writing,
    visual vocabulary learning, translation practice, and more!

    ---
    """)

    with gr.Tabs():

        # ══════════════════════════════════════════════════
        # TAB 1: Writing Feedback
        # ══════════════════════════════════════════════════
        with gr.TabItem("📝 Writing Feedback", id=1):
            gr.Markdown("""
            ### ✍️ Get AI Feedback on Your Writing
            Write a paragraph in your target language and receive detailed
            corrections, explanations, and improvement suggestions.
            """)

            with gr.Row():
                with gr.Column(scale=1):
                    writing_lang = gr.Dropdown(
                        choices=LANGUAGES,
                        value="French",
                        label="Target Language"
                    )
                    writing_level = gr.Dropdown(
                        choices=PROFICIENCY_LEVELS,
                        value="Intermediate (B1-B2)",
                        label="Proficiency Level"
                    )
                with gr.Column(scale=2):
                    writing_input = gr.Textbox(
                        label="Your Text",
                        placeholder="Write something in your target language...",
                        lines=6
                    )

            writing_btn = gr.Button("📝 Get Feedback", variant="primary")
            writing_output = gr.Markdown(label="Feedback")
            writing_time = gr.Textbox(label="Performance", interactive=False)

            # Example inputs
            gr.Examples(
                examples=[
                    ["Je suis allé au magazin hier pour acheter du pain. Le pain était très bon et je mange beaucoup.", "French", "Intermediate (B1-B2)"],
                    ["Ninajifunza Kiswahili kwa sababu ninapenda lugha ya Afrika. Mimi ni mwanafunzi mzuri.", "Swahili", "Beginner (A1-A2)"],
                    ["きのう友達とレストランに食べました。とても美味しいでした。", "Japanese", "Beginner (A1-A2)"],
                ],
                inputs=[writing_input, writing_lang, writing_level]
            )

            writing_btn.click(
                fn=writing_feedback,
                inputs=[writing_input, writing_lang, writing_level],
                outputs=[writing_output, writing_time]
            )

        # ══════════════════════════════════════════════════
        # TAB 2: Visual Learning (Image Recognition)
        # ══════════════════════════════════════════════════
        with gr.TabItem("🖼️ Visual Learning", id=2):
            gr.Markdown("""
            ### 📸 Learn Vocabulary Through Images
            Upload any image — the AI will identify objects and teach you
            their names in your target language with pronunciation and example sentences.
            """)

            with gr.Row():
                with gr.Column(scale=1):
                    visual_image = gr.Image(
                        label="Upload an Image",
                        type="pil",
                        height=300
                    )
                    visual_target = gr.Dropdown(
                        choices=LANGUAGES,
                        value="French",
                        label="Learn Words In"
                    )
                    visual_source = gr.Dropdown(
                        choices=LANGUAGES,
                        value="English",
                        label="Translate From"
                    )
                    visual_btn = gr.Button("🔍 Identify & Translate", variant="primary")

                with gr.Column(scale=2):
                    visual_output = gr.Markdown(label="Visual Vocabulary")
                    visual_time = gr.Textbox(label="Performance", interactive=False)

            visual_btn.click(
                fn=visual_learning,
                inputs=[visual_image, visual_target, visual_source],
                outputs=[visual_output, visual_time]
            )

        # ══════════════════════════════════════════════════
        # TAB 3: Translation Practice
        # ══════════════════════════════════════════════════
        with gr.TabItem("🔄 Translation Practice", id=3):
            gr.Markdown("""
            ### 🔄 Practice Translating & Get Scored
            Translate a sentence and the AI will compare your translation
            to an ideal one, showing you exactly what to improve.
            """)

            with gr.Row():
                trans_source_lang = gr.Dropdown(
                    choices=LANGUAGES,
                    value="English",
                    label="From Language"
                )
                trans_target_lang = gr.Dropdown(
                    choices=LANGUAGES,
                    value="French",
                    label="To Language"
                )
                trans_level = gr.Dropdown(
                    choices=PROFICIENCY_LEVELS,
                    value="Intermediate (B1-B2)",
                    label="Level"
                )

            trans_original = gr.Textbox(
                label="Original Text (to translate)",
                placeholder="Enter the text to translate...",
                lines=3
            )
            trans_student = gr.Textbox(
                label="Your Translation",
                placeholder="Write your translation here...",
                lines=3
            )

            trans_btn = gr.Button("✅ Check My Translation", variant="primary")
            trans_output = gr.Markdown(label="Translation Feedback")
            trans_time = gr.Textbox(label="Performance", interactive=False)

            gr.Examples(
                examples=[
                    ["The children are playing in the park near the river.", "Les enfants jouent dans le parc près de la rivière.", "English", "French", "Intermediate (B1-B2)"],
                    ["I would like to book a table for two people tonight.", "Je voudrais réserver une table pour deux personnes ce soir.", "English", "French", "Beginner (A1-A2)"],
                ],
                inputs=[trans_original, trans_student, trans_source_lang, trans_target_lang, trans_level]
            )

            trans_btn.click(
                fn=translation_practice,
                inputs=[trans_original, trans_student, trans_source_lang, trans_target_lang, trans_level],
                outputs=[trans_output, trans_time]
            )

        # ══════════════════════════════════════════════════
        # TAB 4: Vocabulary Builder
        # ══════════════════════════════════════════════════
        with gr.TabItem("📚 Vocabulary Builder", id=4):
            gr.Markdown("""
            ### 📖 Build Your Vocabulary by Topic
            Choose any topic and get a complete vocabulary lesson with words,
            sentences, phrases, and a quiz to test yourself.
            """)

            with gr.Row():
                vocab_topic = gr.Textbox(
                    label="Topic",
                    placeholder="e.g., Food, Travel, School, Weather, Technology...",
                    scale=2
                )
                vocab_lang = gr.Dropdown(
                    choices=LANGUAGES,
                    value="Swahili",
                    label="Target Language",
                    scale=1
                )
                vocab_level = gr.Dropdown(
                    choices=PROFICIENCY_LEVELS,
                    value="Beginner (A1-A2)",
                    label="Level",
                    scale=1
                )

            vocab_num = gr.Slider(
                minimum=5, maximum=20, value=10, step=1,
                label="Number of Words"
            )

            vocab_btn = gr.Button("📚 Generate Lesson", variant="primary")
            vocab_output = gr.Markdown(label="Vocabulary Lesson")
            vocab_time = gr.Textbox(label="Performance", interactive=False)

            gr.Examples(
                examples=[
                    ["Food and Cooking", "Swahili", "Beginner (A1-A2)"],
                    ["School and Education", "French", "Intermediate (B1-B2)"],
                    ["Transportation", "Japanese", "Beginner (A1-A2)"],
                    ["Family and Relationships", "Lingala", "Beginner (A1-A2)"],
                ],
                inputs=[vocab_topic, vocab_lang, vocab_level]
            )

            vocab_btn.click(
                fn=vocabulary_builder,
                inputs=[vocab_topic, vocab_lang, vocab_level, vocab_num],
                outputs=[vocab_output, vocab_time]
            )

        # ══════════════════════════════════════════════════
        # TAB 5: Conversation Practice
        # ══════════════════════════════════════════════════
        with gr.TabItem("💬 Conversation Practice", id=5):
            gr.Markdown("""
            ### 🗣️ Practice Real Conversations
            Choose a scenario and chat with the AI in your target language.
            You'll get corrections and tips after each message.
            """)

            with gr.Row():
                convo_lang = gr.Dropdown(
                    choices=LANGUAGES,
                    value="French",
                    label="Practice Language"
                )
                convo_level = gr.Dropdown(
                    choices=PROFICIENCY_LEVELS,
                    value="Beginner (A1-A2)",
                    label="Level"
                )
                convo_scenario = gr.Dropdown(
                    choices=[
                        "At a restaurant ordering food",
                        "Asking for directions on the street",
                        "Job interview at a tech company",
                        "Meeting a new friend at school",
                        "Shopping at a market",
                        "Visiting a doctor",
                        "Checking into a hotel",
                        "At the train station buying tickets",
                    ],
                    value="At a restaurant ordering food",
                    label="Conversation Scenario"
                )

            convo_history = gr.State(value="")  # Hidden state for chat history

            convo_input = gr.Textbox(
                label="Your Message",
                placeholder="Type in your target language... (e.g., 'Bonjour, je voudrais...')",
                lines=2
            )

            with gr.Row():
                convo_btn = gr.Button("💬 Send Message", variant="primary")
                convo_reset = gr.Button("🔄 New Conversation", variant="secondary")

            convo_output = gr.Markdown(label="AI Response & Notes")
            convo_time = gr.Textbox(label="Performance", interactive=False)

            convo_btn.click(
                fn=conversation_practice,
                inputs=[convo_input, convo_scenario, convo_lang, convo_level, convo_history],
                outputs=[convo_output, convo_history, convo_time]
            )

            # Reset conversation
            convo_reset.click(
                fn=lambda: ("", "", ""),
                outputs=[convo_output, convo_history, convo_time]
            )

    # ── Footer ────────────────────────────────────────────
    gr.Markdown("""
    ---
    ### ℹ️ About This Tool

    **Mwalimu-LangLearn** ("Mwalimu" means "Teacher" in Swahili) is an AI-powered
    language learning platform built with:
    - 🧠 **Google Gemma 3n** (E2B-IT) — Compact multimodal AI model
    - 🖼️ **MobileNet-V5** vision encoder for image understanding
    - 🌐 **140+ language** support for multilingual learning
    - 🎨 **Gradio** for interactive web interface

    Built by **Pascal Burume Buhendwa** | ABE Initiative @ Kobe Institute of Computing

    *Part of the Mwalimu-STEM-GenAI research project for AI-powered education in the DRC*
    """)

print("✅ Gradio interface built successfully!")

/tmp/ipython-input-797585069.py:20: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-797585069.py:20: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(


✅ Gradio interface built successfully!


---
## Step 8: Launch the Application

Launch the Gradio app. On Kaggle, this creates a shareable public URL that lasts for the notebook session.

**Key parameters:**
- `share=True` — Creates a public URL (needed on Kaggle)
- `server_name="0.0.0.0"` — Listens on all interfaces
- `server_port=7860` — Default Gradio port

In [11]:
# ============================================================
# STEP 8: Launch the Application
# ============================================================

# Launch with a shareable public link
demo.launch(
    share=True,           # Creates a public URL (required for Kaggle)
    server_name="0.0.0.0",
    server_port=7860,
    show_error=True,      # Show errors in the UI for debugging
)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fbfb48f8a47d5420cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---
## 📋 Summary: Step-by-Step Implementation Guide

### What We Built:

| Step | What | Why |
|------|------|-----|
| **1** | Install dependencies | Gemma 3n needs transformers ≥4.53.0, timm for vision |
| **2** | Import libraries | Load all required modules |
| **3** | Download & load model | Use kagglehub for Kaggle, or HuggingFace directly |
| **4** | Core inference functions | `query_model()` for text, `query_model_with_image()` for multimodal |
| **5** | Feature functions | 5 learning modes with specialized system prompts |
| **6** | Test features | Verify each feature before building UI |
| **7** | Build Gradio UI | 5-tab interface with examples and state management |
| **8** | Launch app | Public URL for sharing |

### How Gemma 3n Works Under the Hood:

1. **Text Input** → Tokenized by `AutoProcessor` → Fed to language decoder
2. **Image Input** → Resized to 256/512/768px → Encoded by MobileNet-V5 to 256 tokens → Concatenated with text tokens
3. **Generation** → Autoregressive decoding with temperature sampling → Decoded back to text

### Key Technical Details:

- **Chat Template**: Gemma 3n uses `<start_of_turn>user\n...\n<end_of_turn>` format
- **System Prompt**: Set via `{"role": "system"}` message — controls the model's behavior
- **Image Processing**: The processor handles all resizing and normalization
- **`disable_compile=True`**: Required for Kaggle environments to avoid JIT compilation issues
- **`device_map="auto"`**: Automatically places model layers on available GPU(s)

---

### 🚀 Next Steps for Enhancement:

1. **Audio Input**: Gemma 3n supports audio (USM encoder) — add speech recognition for pronunciation practice
2. **Fine-tuning**: Fine-tune on language learning datasets for better pedagogical responses
3. **Offline Deployment**: Use LiteRT (TFLite) format for mobile/edge deployment
4. **User Progress Tracking**: Add a database to track vocabulary mastery over time
5. **Spaced Repetition**: Implement SRS (like Anki) for vocabulary review scheduling

---

### 📚 References

- [Gemma 3n Model Card (Kaggle)](https://www.kaggle.com/models/google/gemma-3n)
- [Gemma 3n on HuggingFace](https://huggingface.co/google/gemma-3n-E2B-it)
- [How to use Gemma 3n on Kaggle (Paul Timothy Mooney)](https://www.kaggle.com/code/paultimothymooney/how-to-use-gemma-3n-on-kaggle)
- [Gemma 3n Developer Guide (Google)](https://developers.googleblog.com/en/introducing-gemma-3n-developer-guide/)
- [HuggingFace Gemma 3n Blog](https://huggingface.co/blog/gemma3n)
- [Gradio Documentation](https://www.gradio.app/docs)